In [1]:
!pip install -q langchain langchain_community langchain-google-genai langchain_experimental sentence-transformers langchain_chroma langchainhub unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('/kaggle/input/avinash-resume/Avinash_Resume.pdf')
data = loader.load()

In [3]:
data

[Document(metadata={'source': '/kaggle/input/avinash-resume/Avinash_Resume.pdf', 'page': 0}, page_content='Avinash R\n+91-9629149765 ravinash218@gmail.com LinkedIn GitHub Portfolio Hashnode\nEducation\nMepco Schlenk Engineering College\nBachelor of Engineering (B.Eng) in Computer Science And Engineering (CSE) CGPA: 8.79\nWork Experience\nAugrade Bangalore\nSoftware Engineer - Machine Learning Aug 2023 - Present\n• Dreamspace: Geometric Graph Based Building Information Modeling System| Python, Git, IfcOpenShell\n∗ Orchestrated the establishment of a system that processes architectural CAD files to output a comprehensive\ngeometric graph based model of buildings.\n∗ Engineered sophisticated algorithms to autonomously associate building systems, including structural and MEP\n(mechanical, electrical, and plumbing) into a unified and fully functional model.\n∗ Leveraged IfcOpenShell to create a Building Information Modeling (BIM) Industry Foundation Classes (IFC) file\nfrom the geometric gr

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print(len(docs))

6


In [7]:
docs[0]

Document(metadata={'source': '/kaggle/input/avinash-resume/Avinash_Resume.pdf', 'page': 0}, page_content='Avinash R\n+91-9629149765 ravinash218@gmail.com LinkedIn GitHub Portfolio Hashnode\nEducation\nMepco Schlenk Engineering College\nBachelor of Engineering (B.Eng) in Computer Science And Engineering (CSE) CGPA: 8.79\nWork Experience\nAugrade Bangalore\nSoftware Engineer - Machine Learning Aug 2023 - Present\n• Dreamspace: Geometric Graph Based Building Information Modeling System| Python, Git, IfcOpenShell\n∗ Orchestrated the establishment of a system that processes architectural CAD files to output a comprehensive\ngeometric graph based model of buildings.\n∗ Engineered sophisticated algorithms to autonomously associate building systems, including structural and MEP\n(mechanical, electrical, and plumbing) into a unified and fully functional model.\n∗ Leveraged IfcOpenShell to create a Building Information Modeling (BIM) Industry Foundation Classes (IFC) file\nfrom the geometric gra

In [8]:
import os
GOOGLE_API_KEY = 'xxxxxxxxxxxxxxxxxx'
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [20]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [22]:
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [23]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [28]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [29]:
retrieved_docs = retriever.invoke("Who is Avinash?")
print(len(retrieved_docs))

3


In [30]:
retrieved_docs

[Document(metadata={'page': 0, 'source': '/kaggle/input/avinash-resume/Avinash_Resume.pdf'}, page_content='Avinash R\n+91-9629149765 ravinash218@gmail.com LinkedIn GitHub Portfolio Hashnode\nEducation\nMepco Schlenk Engineering College\nBachelor of Engineering (B.Eng) in Computer Science And Engineering (CSE) CGPA: 8.79\nWork Experience\nAugrade Bangalore\nSoftware Engineer - Machine Learning Aug 2023 - Present\n• Dreamspace: Geometric Graph Based Building Information Modeling System| Python, Git, IfcOpenShell\n∗ Orchestrated the establishment of a system that processes architectural CAD files to output a comprehensive\ngeometric graph based model of buildings.\n∗ Engineered sophisticated algorithms to autonomously associate building systems, including structural and MEP\n(mechanical, electrical, and plumbing) into a unified and fully functional model.\n∗ Leveraged IfcOpenShell to create a Building Information Modeling (BIM) Industry Foundation Classes (IFC) file\nfrom the geometric gr

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro', temperature=0.3, max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
query='Who is Avinash?'
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash R is a Software Engineer at Augrade in Bangalore, specializing in Machine Learning.  He holds a B.Eng in Computer Science and Engineering from Mepco Schlenk Engineering College, graduating with a CGPA of 8.79.  His experience includes developing a geometric graph-based building information modeling system and contributing to various open-source projects involving machine learning, deep learning, and parallel computing.



In [35]:
query='How to contact Avinash?'
response = rag_chain.invoke({"input":query})
print(response['answer'])

You can contact Avinash R by phone at +91-9629149765 or by email at ravinash218@gmail.com.  He also has a LinkedIn profile and GitHub and Hashnode portfolios, though the links are not provided.



In [39]:
query="What is Avinash's skillset?"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash's technical skills include machine learning, parallel computing with CUDA, deep learning, LLMs, MongoDB, and CI/CD.  He is proficient in Python, C, C++, Java, and SQL, and utilizes frameworks like Pytorch, Keras, Sci-Kit Learn, Flask, and FastAPI.  He also has experience with various tools, including Git, AWS, GCP, Docker, and VS Code.



In [40]:
query="What is Avinash's role and what are the companies in which he worked / working?"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash is a Software Engineer - Machine Learning. He currently works at Augrade in Bangalore.  He previously worked at Intelligence Concepts, but his role there is not specified.



In [41]:
query="Tell me about Avinash's open source contributions"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash has made several open-source contributions spanning diverse topics.  These include projects focused on Heterogeneous Parallel Programming, Machine Learning/Deep Learning (including Generative Adversarial Networks), and fundamental Computer Science concepts like Data Structures, Algorithms, and Operating Systems.  His contributions highlight expertise in parallel computing, GPU acceleration, and various machine learning domains.



In [42]:
query="Does Avinash's familiar with CUDA programming and CI CD pipelines?"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Yes, Avinash lists Parallel Computing with CUDA and CI/CD as technical skills.  His projects also mention CUDA and Docker, which is commonly used in CI/CD pipelines. He also has experience with Git, AWS, GCP, and Kubernetes, which are relevant to CI/CD.



In [45]:
query="What would you suggest whether I should recruit him as Machine learning Engineer? Answer this based on his most recent experience"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Based on his recent experience at Augrade, he has experience building geometric graph-based BIM systems, using relevant tools like IfcOpenShell and Python.  His internship also involved image segmentation using PyTorch and TensorFlow, achieving high accuracy.  Therefore, he appears qualified for a machine learning engineering role.



In [47]:
query="Give me reasons why I should not recruit Avinash for Machine Learning Engineer position?"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash's resume focuses on software engineering and BIM, with limited specific machine learning project details beyond "AI Playground."  The resume lacks quantifiable achievements or metrics demonstrating successful ML model development and deployment.  Therefore, it's unclear if Avinash possesses the required depth of experience for a dedicated Machine Learning Engineer role.



In [48]:
query="Based on the resume contents, How can Avinash increase his technical skills to get one of the best job in Machine Learning. How should he equip himself?"
response = rag_chain.invoke({"input":query})
print(response['answer'])

Avinash has a solid foundation in machine learning and related tools.  He could deepen his expertise by focusing on advanced topics like reinforcement learning, distributed computing frameworks (e.g., Spark), and MLOps.  Contributing to more complex open-source projects and gaining experience with large-scale datasets would also enhance his profile.



In [53]:
query="List names of technology that convey Avinash's skills in Machine Learning and AI. Give in bulletin"
response = rag_chain.invoke({"input":query})
print(response['answer'])

* Machine Learning
* Deep Learning
* LLM
* Pytorch
* Keras
* Sci-Kit Learn
* Tensorflow
* Generative Adversarial Networks (GANs)

